# Spark
## Tools for NLP

In [30]:
import findspark

from pyspark import SparkConf

from pyspark.sql import SparkSession
import pyspark.sql.functions as F

from pyspark.ml import Pipeline

from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import RegexTokenizer
from pyspark.ml.feature import StopWordsRemover

from pyspark.sql.types import IntegerType

from matplotlib import pyplot as plt
import seaborn as sns

In [31]:
findspark.init()
findspark.find()

%matplotlib inline
sns.set_theme(style='darkgrid')
sns.set_context("notebook", rc={"lines.linewidth": 2.5})

In [32]:
random_seed = 0

In [33]:
conf = SparkConf() \
    .setAppName('nlp') \
    #.setMaster('local') \
    #.set('spark.executor.memory', '8g') \
    #.set('spark.driver.maxResultSize', '8g') \
    #.set("spark.memory.fraction", "0.6") \
    #.set("spark.memory.storageFraction", "0.5") \
    #.set("spark.sql.shuffle.partitions", "5") \
    #.set("spark.memory.offHeap.enabled", "false") \
    #.set("spark.reducer.maxSizeInFlight", "96m") \
    #.set("spark.shuffle.file.buffer", "256k") \
    #.set("spark.sql.debug.maxToStringFields", "100") \
    #.set('spark.sql.autoBroadcastJoinThreshold', '-1')

In [34]:
%%capture

spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [35]:
sen_df = spark.createDataFrame(
    [
    (0, 'Hi I heard about Spark'),
    (1, 'I wish Java could use case classes'),
    (2, 'Logistic,regression,models,are,neat')
    ],
    ['id', 'sentence']
)

In [36]:
sen_df.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|Hi I heard about ...|
|  1|I wish Java could...|
|  2|Logistic,regressi...|
+---+--------------------+



In [37]:
Tokenizer?

Init signature:
Tokenizer(
    *,
    inputCol: Optional[str] = None,
    outputCol: Optional[str] = None,
)
Docstring:     
A tokenizer that converts the input string to lowercase and then
splits it by white spaces.

.. versionadded:: 1.3.0

Examples
--------
>>> df = spark.createDataFrame([("a b c",)], ["text"])
>>> tokenizer = Tokenizer(outputCol="words")
>>> tokenizer.setInputCol("text")
Tokenizer...
>>> tokenizer.transform(df).head()
Row(text='a b c', words=['a', 'b', 'c'])
>>> # Change a parameter.
>>> tokenizer.setParams(outputCol="tokens").transform(df).head()
Row(text='a b c', tokens=['a', 'b', 'c'])
>>> # Temporarily modify a parameter.
>>> tokenizer.transform(df, {tokenizer.outputCol: "words"}).head()
Row(text='a b c', words=['a', 'b', 'c'])
>>> tokenizer.transform(df).head()
Row(text='a b c', tokens=['a', 'b', 'c'])
>>> # Must use keyword arguments to specify params.
>>> tokenizer.setParams("text")
Traceback (most recent call last):
    ...
TypeError: Method setParams force

In [38]:
tokenizer = Tokenizer(
    inputCol='sentence',
    outputCol='words'
)

In [39]:
RegexTokenizer?

Init signature:
RegexTokenizer(
    *,
    minTokenLength: int = 1,
    gaps: bool = True,
    pattern: str = '\\s+',
    inputCol: Optional[str] = None,
    outputCol: Optional[str] = None,
    toLowercase: bool = True,
)
Docstring:     
A regex based tokenizer that extracts tokens either by using the
provided regex pattern (in Java dialect) to split the text
(default) or repeatedly matching the regex (if gaps is false).
Optional parameters also allow filtering tokens using a minimal
length.
It returns an array of strings that can be empty.

.. versionadded:: 1.4.0

Examples
--------
>>> df = spark.createDataFrame([("A B  c",)], ["text"])
>>> reTokenizer = RegexTokenizer()
>>> reTokenizer.setInputCol("text")
RegexTokenizer...
>>> reTokenizer.setOutputCol("words")
RegexTokenizer...
>>> reTokenizer.transform(df).head()
Row(text='A B  c', words=['a', 'b', 'c'])
>>> # Change a parameter.
>>> reTokenizer.setParams(outputCol="tokens").transform(df).head()
Row(text='A B  c', tokens=['a', 'b'

In [40]:
regex_tokenizer = RegexTokenizer(
    inputCol='sentence',
    outputCol='words',
    pattern='\\W'
)

In [41]:
F.udf?

Signature:
F.udf(
    f: Union[Callable[..., Any], ForwardRef('DataTypeOrString'), NoneType] = None,
    returnType: 'DataTypeOrString' = StringType(),
    *,
    useArrow: Optional[bool] = None,
) -> Union[ForwardRef('UserDefinedFunctionLike'), Callable[[Callable[..., Any]], ForwardRef('UserDefinedFunctionLike')]]
Docstring:
Creates a user defined function (UDF).

.. versionadded:: 1.3.0

.. versionchanged:: 3.4.0
    Supports Spark Connect.

Parameters
----------
f : function
    python function if used as a standalone function
returnType : :class:`pyspark.sql.types.DataType` or str
    the return type of the user-defined function. The value can be either a
    :class:`pyspark.sql.types.DataType` object or a DDL-formatted type string.
useArrow : bool or None
    whether to use Arrow to optimize the (de)serialization. When it is None, the
    Spark config "spark.sql.execution.pythonUDF.arrow.enabled" takes effect.

Examples
--------
>>> from pyspark.sql.types import IntegerType
>>> sl

In [42]:
count_tokens = F.udf(lambda words : len(words), IntegerType())

In [43]:
tokendized = tokenizer.transform(sen_df)

In [44]:
tokendized.show()

+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|
|  1|I wish Java could...|[i, wish, java, c...|
|  2|Logistic,regressi...|[logistic,regress...|
+---+--------------------+--------------------+



In [45]:
tokendized.withColumn('tokens', count_tokens(F.col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish Java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic,regress...|     1|
+---+--------------------+--------------------+------+



In [46]:
rg_tokenized = regex_tokenizer.transform(sen_df).withColumn('tokens', count_tokens(F.col('words')))
rg_tokenized.show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish Java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic, regres...|     5|
+---+--------------------+--------------------+------+



In [47]:
sentenceData = spark.createDataFrame(
    [
        (0, ["I", "saw", "the", "red", "balloon"]),
        (1, ["Mary", "had", "a", "little", "lamb"])
    ],
    ["id", "raw"])

sentenceData.show()

+---+--------------------+
| id|                 raw|
+---+--------------------+
|  0|[I, saw, the, red...|
|  1|[Mary, had, a, li...|
+---+--------------------+



In [48]:
StopWordsRemover?

Init signature:
StopWordsRemover(
    *,
    inputCol: Optional[str] = None,
    outputCol: Optional[str] = None,
    stopWords: Optional[List[str]] = None,
    caseSensitive: bool = False,
    locale: Optional[str] = None,
    inputCols: Optional[List[str]] = None,
    outputCols: Optional[List[str]] = None,
)
Docstring:     
A feature transformer that filters out stop words from input.
Since 3.0.0, :py:class:`StopWordsRemover` can filter out multiple columns at once by setting
the :py:attr:`inputCols` parameter. Note that when both the :py:attr:`inputCol` and
:py:attr:`inputCols` parameters are set, an Exception will be thrown.

.. versionadded:: 1.6.0

Notes
-----
null values from input array are preserved unless adding null to stopWords explicitly.

Examples
--------
>>> df = spark.createDataFrame([(["a", "b", "c"],)], ["text"])
>>> remover = StopWordsRemover(stopWords=["b"])
>>> remover.setInputCol("text")
StopWordsRemover...
>>> remover.setOutputCol("words")
StopWordsRemover...
>

In [49]:
remover = StopWordsRemover(
    inputCol='raw',
    outputCol='filtered',
)

In [50]:
remover.transform(sentenceData).show()

+---+--------------------+--------------------+
| id|                 raw|            filtered|
+---+--------------------+--------------------+
|  0|[I, saw, the, red...| [saw, red, balloon]|
|  1|[Mary, had, a, li...|[Mary, little, lamb]|
+---+--------------------+--------------------+



## N-gram

In [51]:
from pyspark.ml.feature import NGram

In [55]:
NGram?

Init signature:
NGram(
    *,
    n: int = 2,
    inputCol: Optional[str] = None,
    outputCol: Optional[str] = None,
)
Docstring:     
A feature transformer that converts the input array of strings into an array of n-grams. Null
values in the input array are ignored.
It returns an array of n-grams where each n-gram is represented by a space-separated string of
words.
When the input is empty, an empty array is returned.
When the input array length is less than n (number of elements per n-gram), no n-grams are
returned.

.. versionadded:: 1.5.0

Examples
--------
>>> df = spark.createDataFrame([Row(inputTokens=["a", "b", "c", "d", "e"])])
>>> ngram = NGram(n=2)
>>> ngram.setInputCol("inputTokens")
NGram...
>>> ngram.setOutputCol("nGrams")
NGram...
>>> ngram.transform(df).head()
Row(inputTokens=['a', 'b', 'c', 'd', 'e'], nGrams=['a b', 'b c', 'c d', 'd e'])
>>> # Change n-gram length
>>> ngram.setParams(n=4).transform(df).head()
Row(inputTokens=['a', 'b', 'c', 'd', 'e'], nGrams=['a b c 

In [52]:
wordDataFrame = spark.createDataFrame([
    (0, ["Hi", "I", "heard", "about", "Spark"]),
    (1, ["I", "wish", "Java", "could", "use", "case", "classes"]),
    (2, ["Logistic", "regression", "models", "are", "neat"])
], ["id", "words"])

wordDataFrame.show()

+---+--------------------+
| id|               words|
+---+--------------------+
|  0|[Hi, I, heard, ab...|
|  1|[I, wish, Java, c...|
|  2|[Logistic, regres...|
+---+--------------------+



In [104]:
ngram = NGram(
    n=2,
    inputCol="words",
    outputCol="ngrams"
)

In [105]:

ngramDataFrame = ngram.transform(wordDataFrame)
ngramDataFrame.select("ngrams").show(truncate=False)

+------------------------------------------------------------------+
|ngrams                                                            |
+------------------------------------------------------------------+
|[Hi I, I heard, heard about, about Spark]                         |
|[I wish, wish Java, Java could, could use, use case, case classes]|
|[Logistic regression, regression models, models are, are neat]    |
+------------------------------------------------------------------+



__________________

In [106]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer


In [112]:
sentenceData = spark.createDataFrame(
    [
    (0.0, 'Hi I heard about Spark'),
    (0.0, 'I wish Java could use case classes'),
    (1.0, 'Logistic regression models are neat')
    ],
    ['label', 'sentence']
)
sentenceData.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|  0.0|Hi I heard about ...|
|  0.0|I wish Java could...|
|  1.0|Logistic regressi...|
+-----+--------------------+



In [113]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [116]:
words_data = tokenizer.transform(sentenceData)
words_data.show(truncate=False)

+-----+-----------------------------------+------------------------------------------+
|label|sentence                           |words                                     |
+-----+-----------------------------------+------------------------------------------+
|0.0  |Hi I heard about Spark             |[hi, i, heard, about, spark]              |
|0.0  |I wish Java could use case classes |[i, wish, java, could, use, case, classes]|
|1.0  |Logistic regression models are neat|[logistic, regression, models, are, neat] |
+-----+-----------------------------------+------------------------------------------+



In [117]:
hashing_tf = HashingTF(
    inputCol='words',
    outputCol='rawFeatures'
)

In [119]:
featurized_data = hashing_tf.transform(words_data)
featurized_data.show(truncate=False)

+-----+-----------------------------------+------------------------------------------+------------------------------------------------------------------------------------+
|label|sentence                           |words                                     |rawFeatures                                                                         |
+-----+-----------------------------------+------------------------------------------+------------------------------------------------------------------------------------+
|0.0  |Hi I heard about Spark             |[hi, i, heard, about, spark]              |(262144,[18700,19036,33808,66273,173558],[1.0,1.0,1.0,1.0,1.0])                     |
|0.0  |I wish Java could use case classes |[i, wish, java, could, use, case, classes]|(262144,[19036,20719,55551,58672,98717,109547,192310],[1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|1.0  |Logistic regression models are neat|[logistic, regression, models, are, neat] |(262144,[46243,58267,91006,160975,190884],[1.0,1.0,1.0

In [121]:
idf = IDF(
    inputCol='rawFeatures',
    outputCol='features'
)

In [124]:
idf_model = idf.fit(featurized_data)

In [125]:
rescaled_data = idf_model.transform(featurized_data)

In [128]:
rescaled_data.select('label', 'features').show(truncate=False)

23/11/10 14:03:22 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB
23/11/10 14:03:23 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB


+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                                                                      |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0.0  |(262144,[18700,19036,33808,66273,173558],[0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453])                                                   |
|0.0  |(262144,[19036,20719,55551,58672,98717,109547,192310],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])|
|1.0 

23/11/10 14:03:23 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB


In [129]:
from pyspark.ml.feature import CountVectorizer

In [131]:
df = spark.createDataFrame(
    [
      (0, 'a b c'.split(' '))  ,
      (1, 'a b b c a'.split(' ')),
    ],
    ['id', 'words']
)
df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [133]:
cv = CountVectorizer(
    inputCol='words',
    outputCol='features',
    vocabSize=3,
    minDF=2.0
)

In [135]:
result = cv.fit(df).transform(df)
result.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

